# End of Term project
## Running SpeechBrain-MOABB data with a Simpler Model, SimpleEEGNEt
Reasearch Question: Can a model with a simpler architecture achieve comparable accuracy to EEGNet in a shorter training time? In response to this question, I developed a new model named SimpleEEGNet.<br>
"EEGNet vs. SimpleEEGNet Comparison:

Architecture:<br>
* EEGNet: Complex, utilizing various convolution types and pooling.<br>
* SimpleEEGNet: Simplified, featuring two convolutional layers and average pooling.<br>

Fully Connected Layers:<br>
* EEGNet: Single layer.<br>
* SimpleEEGNet: Three layers, offering more depth.<br>

Dropout:<br>
* EEGNet: After each pooling layer.<br>
* SimpleEEGNet: After the second convolutional layer.<br>

Implementation:<br>
* EEGNet: SpeechBrain library.<br>
* SimpleEEGNet: Direct PyTorch implementation.<br>

Activation Functions:<br>
* Both models support various functions.<br>

Output Layer:<br>
* Both use LogSoftmax for classification.<br>

## **Prerequisites**


### Download SpeechBrain-MOABB

SpeechBrain-MOABB can be downloaded from the GitHub repository listed below.

In [ ]:
%%capture
!git clone https://github.com/speechbrain/benchmarks.git
%cd benchmarks
!git submodule update --init --recursive
%cd speechbrain
!pip install -r requirements.txt
!pip install -e .
%cd /content/benchmarks/benchmarks/MOABB
!pip install -r ../../requirements.txt    # Install base dependencies
!pip install -r extra-requirements.txt    # Install additional dependencies
%cd /content/benchmarks/benchmarks/MOABB
%env PYTHON_PATH=/content/benchmarks/

### Save my own implemented model and its yaml file

Run the two next cells below to save my own implemented model(SimpleEEGNet.py) in /content/benchmarks/benchmarks/MOABB/models/<br>
and its yaml file (SimpleEEGNet.yaml) in /content/benchmarks/benchmarks/MOABB/hparams/MotorImagery/BNCI2014001/

In [ ]:
#CREATE SimpleEEGNet.yaml file
content = """
seed: 1234
__set_torchseed: !apply:torch.manual_seed [!ref <seed>]

# DIRECTORIES
data_folder: !PLACEHOLDER  #'/path/to/dataset'. The dataset will be automatically downloaded in this folder
cached_data_folder: !PLACEHOLDER #'path/to/pickled/dataset'
output_folder: !PLACEHOLDER #'path/to/results'

# DATASET HPARS
# Defining the MOABB dataset.
dataset: !new:moabb.datasets.BNCI2014001
save_prepared_dataset: True
data_iterator_name: 'leave-one-session-out'
target_subject_idx: 0
target_session_idx: 1
events_to_load: null
original_sample_rate: 250
sample_rate: 125
fmin: 1
fmax: 40
n_classes: 4
tmin: 0.
tmax: 4.0
n_steps_channel_selection: 3
T: !apply:math.ceil
    - !ref <sample_rate> * (<tmax> - <tmin>)
C: 22
test_with: 'best'
test_key: "acc"

# METRICS
f1: !name:sklearn.metrics.f1_score
    average: 'macro'
acc: !name:sklearn.metrics.balanced_accuracy_score
cm: !name:sklearn.metrics.confusion_matrix
metrics:
    f1: !ref <f1>
    acc: !ref <acc>
    cm: !ref <cm>

# TRAINING HPARS
n_train_examples: 100
avg_models: 10
number_of_epochs: 1000
lr: 0.0001
max_lr: !ref <lr>
base_lr: 0.00000001
step_size_multiplier: 5
step_size: !apply:round
    - !ref <step_size_multiplier> * <n_train_examples> / <batch_size>
lr_annealing: !new:speechbrain.nnet.schedulers.CyclicLRScheduler
    base_lr: !ref <base_lr>
    max_lr: !ref <max_lr>
    step_size: !ref <step_size>
label_smoothing: 0.0
loss: !name:speechbrain.nnet.losses.nll_loss
    label_smoothing: !ref <label_smoothing>
optimizer: !name:torch.optim.Adam
    lr: !ref <lr>
epoch_counter: !new:speechbrain.utils.epoch_loop.EpochCounter
    limit: !ref <number_of_epochs>
batch_size: 32
valid_ratio: 0.2

# DATA AUGMENTATION
# cutcat (disabled when min_num_segments=max_num_segments=1)
max_num_segments: 3 # @orion_step2: --max_num_segments~"uniform(2, 6, discrete=True)"
cutcat: !new:speechbrain.augment.time_domain.CutCat
    min_num_segments: 2
    max_num_segments: !ref <max_num_segments>
# random amplitude gain between 0.5-1.5 uV (disabled when amp_delta=0.)
amp_delta: 0.01742 # @orion_step2: --amp_delta~"uniform(0.0, 0.5)"
rand_amp: !new:speechbrain.augment.time_domain.RandAmp
    amp_low: !ref 1 - <amp_delta>
    amp_high: !ref 1 + <amp_delta>
# random shifts between -300 ms to 300 ms (disabled when shift_delta=0.)
shift_delta_: 1 # orion_step2: --shift_delta_~"uniform(0, 25, discrete=True)"
shift_delta: !ref 1e-2 * <shift_delta_> # 0.250 # 0.-0.25 with steps of 0.01
min_shift: !apply:math.floor
    - !ref 0 - <sample_rate> * <shift_delta>
max_shift: !apply:math.floor
    - !ref 0 + <sample_rate> * <shift_delta>
time_shift: !new:speechbrain.augment.freq_domain.RandomShift
    min_shift: !ref <min_shift>
    max_shift: !ref <max_shift>
    dim: 1
# injection of gaussian white noise
snr_white_low: 15.0 # @orion_step2: --snr_white_low~"uniform(0.0, 15, precision=2)"
snr_white_delta: 19.1 # @orion_step2: --snr_white_delta~"uniform(5.0, 20.0, precision=3)"
snr_white_high: !ref <snr_white_low> + <snr_white_delta>
add_noise_white: !new:speechbrain.augment.time_domain.AddNoise
    snr_low: !ref <snr_white_low>
    snr_high: !ref <snr_white_high>

repeat_augment: 1 # @orion_step1: --repeat_augment 0
augment: !new:speechbrain.augment.augmenter.Augmenter
    parallel_augment: True
    concat_original: True
    parallel_augment_fixed_bs: True
    repeat_augment: !ref <repeat_augment>
    shuffle_augmentations: True
    min_augmentations: 4
    max_augmentations: 4
    augmentations: [
        !ref <cutcat>,
        !ref <rand_amp>,
        !ref <time_shift>,
        !ref <add_noise_white>]

# DATA NORMALIZATION
dims_to_normalize: 1
normalize: !name:speechbrain.processing.signal_processing.mean_std_norm
    dims: !ref <dims_to_normalize>

# MODEL
input_shape: [null, !ref <T>, !ref <C>, null]
cnn_temporal_kernels: 4
cnn_temporal_kernelsize: 20
cnn_spatial_depth_multiplier: 1
cnn_spatial_pool: 2
activation_type: 'relu'

model: !new:models.SimpleEEGNet.SimpleEEGNet
    input_shape: !ref <input_shape>
    cnn_temporal_kernels: !ref <cnn_temporal_kernels>
    cnn_temporal_kernelsize: [!ref <cnn_temporal_kernelsize>, 1]
    cnn_spatial_depth_multiplier: !ref <cnn_spatial_depth_multiplier>
    cnn_spatial_pool: !ref <cnn_spatial_pool>
    activation_type: !ref <activation_type>
    dense_n_neurons: !ref <n_classes>
"""
with open('/content/benchmarks/benchmarks/MOABB/hparams/MotorImagery/BNCI2014001/SimpleEEGNet.yaml', 'w') as f:
    f.write(content)

In [ ]:
#CREATE SimpleEEGNet.py file
content = """
import torch
import torch.nn as nn

class SimpleEEGNet(nn.Module):
    def __init__(
        self,
        input_shape=None,  # (1, T, C, 1)
        cnn_temporal_kernels=8,
        cnn_temporal_kernelsize=32,
        cnn_spatial_depth_multiplier=2,
        cnn_spatial_pool=4,
        dropout=0.5,
        dense_n_neurons=4,
        activation_type="elu",
    ):
        super().__init__()

        if input_shape is None:
            raise ValueError("Must specify input_shape")

        if activation_type == "gelu":
            activation = nn.GELU()
        elif activation_type == "elu":
            activation = nn.ELU()
        elif activation_type == "relu":
            activation = nn.ReLU()
        elif activation_type == "leaky_relu":
            activation = nn.LeakyReLU()
        elif activation_type == "prelu":
            activation = nn.PReLU()
        else:
            raise ValueError("Wrong hidden activation function")

        # Define CNN layers
        self.conv1 = nn.Conv2d(1, cnn_temporal_kernels, kernel_size=(cnn_temporal_kernelsize, 1))
        self.bn1 = nn.BatchNorm2d(cnn_temporal_kernels)
        self.conv2 = nn.Conv2d(cnn_temporal_kernels, cnn_spatial_depth_multiplier * cnn_temporal_kernels, kernel_size=(1, input_shape[2]))
        self.bn2 = nn.BatchNorm2d(cnn_spatial_depth_multiplier * cnn_temporal_kernels)
        self.activation = activation
        self.pool = nn.AvgPool2d(kernel_size=(cnn_spatial_pool, 1))
        self.dropout = nn.Dropout(dropout)

        # Calculate output shape after convolutional layers
        with torch.no_grad():
            input = torch.ones((1,) + tuple(input_shape[1:-1]) + (1,))
            output = self.forward_conv(input)
            self.num_flat_features = output.view(-1).size(0)

        # Define fully connected layers
        self.fc1 = nn.Linear(self.num_flat_features, dense_n_neurons)
        self.fc2 = nn.Linear(dense_n_neurons, dense_n_neurons)
        self.fc3 = nn.Linear(dense_n_neurons, dense_n_neurons)
        self.fc_out = nn.Linear(dense_n_neurons, dense_n_neurons)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward_conv(self, x):
        x = x.permute(0, 3, 2, 1)
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.activation(x)
        x = self.conv2(x)
        x = self.bn2(x)
        x = self.activation(x)
        x = self.pool(x)
        x = self.dropout(x)
        return x

    def forward(self, x):
        x = self.forward_conv(x)
        x = x.view(-1, self.num_flat_features)
        x = self.fc1(x)
        x = self.activation(x)
        x = self.fc2(x)
        x = self.activation(x)
        x = self.fc3(x)
        x = self.activation(x)
        x = self.fc_out(x)
        x = self.softmax(x)
        return x
"""
with open('/content/benchmarks/benchmarks/MOABB/models/SimpleEEGNet.py', 'w') as f:
    f.write(content)

## **Define a yaml file containing the hyper-parameters defining a decoding pipeline**

Let us address 4-class motor imagery decoding using BNCI2014-001 dataset (also known as "BCI IV2a dataset"), by adopting a leave-one-session-out strategy using the first participants' signals, leaving out the session named '0test'. EEGNet is used as decoder.

You can set all hyper-parameters to specific values if you already know them; otherwise, you can set them to placeholders (i.e., as `!PLACEHOLDER`). For example, folders (e.g., the data folder, the folder for compressed dataset, and the output folder), and dataset information (e.g., data iterator, index of subject and session to use) are usually kept as placeholders.

Before start writing the yaml file, please follow the SpeechBrain tutorial dedicated to HyperPyYAML at https://speechbrain.github.io/tutorial_basics.html.

In [ ]:
from hyperpyyaml import load_hyperpyyaml, dump_hyperpyyaml

example_hyperparams = """
seed: 1234
__set_torchseed: !apply:torch.manual_seed [!ref <seed>]

# DIRECTORIES
data_folder: !PLACEHOLDER  #'/path/to/dataset'. The dataset will be automatically downloaded in this folder
cached_data_folder: !PLACEHOLDER #'path/to/pickled/dataset'
output_folder: !PLACEHOLDER #'path/to/results'

# DATASET HPARS
# Defining the MOABB dataset.
dataset: !new:moabb.datasets.BNCI2014001
save_prepared_dataset: True
data_iterator_name: 'leave-one-session-out'
target_subject_idx: 0
target_session_idx: 1
events_to_load: null
original_sample_rate: 250
sample_rate: 125
fmin: 3.6
fmax: 31.1
n_classes: 4
tmin: 0.
tmax: 4.0
n_steps_channel_selection: 3
T: !apply:math.ceil
    - !ref <sample_rate> * (<tmax> - <tmin>)
C: 22
test_with: 'best'
test_key: "acc"

# METRICS
f1: !name:sklearn.metrics.f1_score
    average: 'macro'
acc: !name:sklearn.metrics.balanced_accuracy_score
cm: !name:sklearn.metrics.confusion_matrix
metrics:
    f1: !ref <f1>
    acc: !ref <acc>
    cm: !ref <cm>

# TRAINING HPARS
n_train_examples: 100
avg_models: 10
number_of_epochs: 900
lr: lr: 0.005
max_lr: !ref <lr>
base_lr: 0.00000001
step_size_multiplier: 5
step_size: !apply:round
    - !ref <step_size_multiplier> * <n_train_examples> / <batch_size>
lr_annealing: !new:speechbrain.nnet.schedulers.CyclicLRScheduler
    base_lr: !ref <base_lr>
    max_lr: !ref <max_lr>
    step_size: !ref <step_size>
label_smoothing: 0.0
loss: !name:speechbrain.nnet.losses.nll_loss
    label_smoothing: !ref <label_smoothing>
optimizer: !name:torch.optim.Adam
    lr: !ref <lr>
epoch_counter: !new:speechbrain.utils.epoch_loop.EpochCounter
    limit: !ref <number_of_epochs>
batch_size: 32
valid_ratio: 0.2

# DATA NORMALIZATION
dims_to_normalize: 1
normalize: !name:speechbrain.processing.signal_processing.mean_std_norm
    dims: !ref <dims_to_normalize>

# MODEL
input_shape: [null, !ref <T>, !ref <C>, null]
cnn_temporal_kernels: 4
cnn_temporal_kernelsize: 20
cnn_spatial_depth_multiplier: 1
cnn_spatial_pool: 2
activation_type: 'relu'

model: !new:models.SimpleEEGNet.SimpleEEGNet
    input_shape: !ref <input_shape>
    cnn_temporal_kernels: !ref <cnn_temporal_kernels>
    cnn_temporal_kernelsize: !ref <cnn_temporal_kernelsize>
    cnn_spatial_depth_multiplier: !ref <cnn_spatial_depth_multiplier>
    cnn_spatial_pool: !ref <cnn_spatial_pool>
    activation_type: !ref <activation_type>
    dense_n_neurons: !ref <n_classes>

"""

# Save the yaml file on disk
f = open('/content/example_hyperparams.yaml', "w")
f.write(example_hyperparams)
f.close()

### **Note about data augmentation**

It is worth highlighting that in the previous yaml file, no data augmentation was included. However, you can easily add data augmentation by defining each augmenter (e.g., applying CutCat and random time shift).

The so-defined augmenters are provided as input to the `Augmenter` class, that will combine and apply the augmenters. For instance, you can perform the augmenters in sequence or in parallel (`parallel_augment` input parameter), use one or more augmenters for augmenting each mini-batch of data (`min_augmentations` and `max_augmentations` input parameters), and repeat data augmentation multiple times for each mini-batch (`repeat_augment` input parameter). See `Augmenter` documentation for further details.

In [ ]:
data_augmentation_hyperparams = """
# DATA AUGMENTATION
# cutcat (disabled when min_num_segments=max_num_segments=1)
max_num_segments: 3 # @orion_step2: --max_num_segments~"uniform(2, 6, discrete=True)"
cutcat: !new:speechbrain.augment.time_domain.CutCat
    min_num_segments: 2
    max_num_segments: !ref <max_num_segments>
# random amplitude gain between 0.5-1.5 uV (disabled when amp_delta=0.)
amp_delta: 0.01742 # @orion_step2: --amp_delta~"uniform(0.0, 0.5)"
rand_amp: !new:speechbrain.augment.time_domain.RandAmp
    amp_low: !ref 1 - <amp_delta>
    amp_high: !ref 1 + <amp_delta>
# random shifts between -300 ms to 300 ms (disabled when shift_delta=0.)
shift_delta_: 1 # orion_step2: --shift_delta_~"uniform(0, 25, discrete=True)"
shift_delta: !ref 1e-2 * <shift_delta_> # 0.250 # 0.-0.25 with steps of 0.01
min_shift: !apply:math.floor
    - !ref 0 - <sample_rate> * <shift_delta>
max_shift: !apply:math.floor
    - !ref 0 + <sample_rate> * <shift_delta>
time_shift: !new:speechbrain.augment.freq_domain.RandomShift
    min_shift: !ref <min_shift>
    max_shift: !ref <max_shift>
    dim: 1
# injection of gaussian white noise
snr_white_low: 15.0 # @orion_step2: --snr_white_low~"uniform(0.0, 15, precision=2)"
snr_white_delta: 19.1 # @orion_step2: --snr_white_delta~"uniform(5.0, 20.0, precision=3)"
snr_white_high: !ref <snr_white_low> + <snr_white_delta>
add_noise_white: !new:speechbrain.augment.time_domain.AddNoise
    snr_low: !ref <snr_white_low>
    snr_high: !ref <snr_white_high>

repeat_augment: 1 # @orion_step1: --repeat_augment 0
augment: !new:speechbrain.augment.augmenter.Augmenter
    parallel_augment: True
    concat_original: True
    parallel_augment_fixed_bs: True
    repeat_augment: !ref <repeat_augment>
    shuffle_augmentations: True
    min_augmentations: 4
    max_augmentations: 4
    augmentations: [
        !ref <cutcat>,
        !ref <rand_amp>,
        !ref <time_shift>,
        !ref <add_noise_white>]
"""

example_hyperparams += data_augmentation_hyperparams

## **Train the neural network on a single cross-validation fold**

Start network training by running the `train.py` script providing the filepath to the yaml file and by overriding the variables set as placeholders in the yaml file. Furthermore, here for brevity we override also the number of training epochs to 50 epochs (instead of 1000 epochs) with `--number_of_epochs 50`. It is worth highlighting that you can also override any other hyper-parameters in the same way.

In [ ]:
import time
# Start time
start_time = time.time()

%cd /content/benchmarks/benchmarks/MOABB/

!python train.py /content/example_hyperparams.yaml \
--data_folder '/content/data/BNCI2014001' \
--cached_data_folder '/content/data' \
--output_folder '/content/results/single-fold-example/BNCI2014001' \
--data_iterator_name 'leave-one-session-out' \
--target_subject_idx 0 \
--target_session_idx 1 \
--number_of_epochs 50 \
--device 'cpu' # Switch to cuda for a speed up.

# End time
end_time = time.time()
# Calculate elapsed time
elapsed_time = end_time - start_time
# Print the elapsed time
print("Elapsed time: {:.4f} seconds".format(elapsed_time))

## **Run a complete experiment by looping over the entire dataset**

In the previous cell, `train.py` was called for a single cross-validation fold (e.g., one participant and one held-out session in leave-one-session-out cross-validation). Thus, we provide a command line interface for easily running training on all participants and cross-validation folds (using `./run_experiments.sh`).

Here, besides the relevant folders, you should specify the hyper-parameter file, the number of participants and sessions to use, the data iteration scheme (leave-one-session-out or leave-one-subject-out). In addition, you can also run the code multiple times, each time with a different random seed used for initializing weights (by setting the `nruns` parameters). Finally, you can define the `device` to use (set to `cpu` if you do not have a GPU).



In [ ]:
# Start time
start_time = time.time()

!./run_experiments.sh --hparams /content/example_hyperparams.yaml \
--data_folder '/content/data/BNCI2014001'\
--cached_data_folder '/content/data' \
--output_folder '/content/results/full-experiment/BNCI2014001' \
--nsbj 9 --nsess 2 --nruns 1 --train_mode 'leave-one-session-out' \
--number_of_epochs 50 \
--device 'cpu'

# End time
end_time = time.time()
# Calculate elapsed time
elapsed_time = end_time - start_time
# Print the elapsed time
print("Elapsed time: {:.4f} seconds".format(elapsed_time))

### Hyperparameter Tuning

In [ ]:
tuned_hyperparams = """
# DATASET HPARS
fmin: 1  # @orion_step1: --fmin~"uniform(0.1, 5, precision=2)"
fmax: 40  # @orion_step1: --fmax~"uniform(20.0, 50.0, precision=3)"

# TRAINING HPARS
number_of_epochs: 1000  # @orion_step1: --number_of_epochs~"uniform(50, 200, discrete=True)"
lr: 0.0001  # @orion_step1: --lr~"choices([0.01, 0.005, 0.001, 0.0005, 0.0001])"

"""

other_hyperparams = """
seed: 1234
__set_torchseed: !apply:torch.manual_seed [!ref <seed>]

# DIRECTORIES
data_folder: !PLACEHOLDER  #'/path/to/dataset'. The dataset will be automatically downloaded in this folder
cached_data_folder: !PLACEHOLDER #'path/to/pickled/dataset'
output_folder: !PLACEHOLDER #'path/to/results'

# DATASET HPARS
# Defining the MOABB dataset.
dataset: !new:moabb.datasets.BNCI2014001
save_prepared_dataset: True
data_iterator_name: 'leave-one-session-out'
target_subject_idx: 0
target_session_idx: 1
events_to_load: null
original_sample_rate: 250
sample_rate: 125
n_classes: 4
tmin: 0.
tmax: 4.0
n_steps_channel_selection: 3
T: !apply:math.ceil
    - !ref <sample_rate> * (<tmax> - <tmin>)
C: 22
test_with: 'best'
test_key: "acc"

# METRICS
f1: !name:sklearn.metrics.f1_score
    average: 'macro'
acc: !name:sklearn.metrics.balanced_accuracy_score
cm: !name:sklearn.metrics.confusion_matrix
metrics:
    f1: !ref <f1>
    acc: !ref <acc>
    cm: !ref <cm>

# TRAINING HPARS
n_train_examples: 100
avg_models: 10
max_lr: !ref <lr>
base_lr: 0.00000001
step_size_multiplier: 5
step_size: !apply:round
    - !ref <step_size_multiplier> * <n_train_examples> / <batch_size>
lr_annealing: !new:speechbrain.nnet.schedulers.CyclicLRScheduler
    base_lr: !ref <base_lr>
    max_lr: !ref <max_lr>
    step_size: !ref <step_size>
label_smoothing: 0.0
loss: !name:speechbrain.nnet.losses.nll_loss
    label_smoothing: !ref <label_smoothing>
optimizer: !name:torch.optim.Adam
    lr: !ref <lr>
epoch_counter: !new:speechbrain.utils.epoch_loop.EpochCounter
    limit: !ref <number_of_epochs>
batch_size: 32
valid_ratio: 0.2

# DATA NORMALIZATION
dims_to_normalize: 1
normalize: !name:speechbrain.processing.signal_processing.mean_std_norm
    dims: !ref <dims_to_normalize>

# MODEL
input_shape: [null, !ref <T>, !ref <C>, null]
cnn_temporal_kernels: 4
cnn_temporal_kernelsize: 20
cnn_spatial_depth_multiplier: 1
cnn_spatial_pool: 2
activation_type: 'relu'

model: !new:models.SimpleEEGNet.SimpleEEGNet
    input_shape: !ref <input_shape>
    cnn_temporal_kernels: !ref <cnn_temporal_kernels>
    cnn_temporal_kernelsize: !ref <cnn_temporal_kernelsize>
    cnn_spatial_depth_multiplier: !ref <cnn_spatial_depth_multiplier>
    cnn_spatial_pool: !ref <cnn_spatial_pool>
    activation_type: !ref <activation_type>
    dense_n_neurons: !ref <n_classes>

"""

sample_hyperparams = tuned_hyperparams + other_hyperparams

In [ ]:
# Save the yaml file on disk
f = open('/content/sample_hyperparams.yaml', "w")
f.write(sample_hyperparams)
f.close()

In [ ]:
%cd /content/benchmarks/benchmarks/MOABB/

!./run_hparam_optimization.sh --hparams '/content/sample_hyperparams.yaml' \
--data_folder '/content/data/BNCI2014001'\
--cached_data_folder '/content/data' \
--output_folder '/content/results/hyperparameter-search/BNCI2014001' \
--nsbj 9 --nsess 2 --nruns 1 --train_mode 'leave-one-session-out' \
--exp_name 'hyperparameter-search' \
--nsbj_hpsearch 1 --nsess_hpsearch 1 \
--nruns_eval 1 \
--eval_metric acc \
--exp_max_trials 5

### Full Training

In [ ]:
#Note: To run the model on the CPU, simply include --device='cpu' in the training command.
!python train.py hparams/MotorImagery/BNCI2014001/SimpleEEGNet.yaml --data_folder=eeg_data --cached_data_folder=eeg_pickled_data --output_folder=results/MotorImagery/BNCI2014001/ --target_subject_idx=0 --target_session_idx=1 --data_iterator_name=leave-one-session-out

##Observations

Train the neural network on a single cross-validation fold takes 2minutes for EEGNet on CPU for EEGNet vs <br>
Run a complete experiment by looping over the entire dataset takes 33minutes on CPU vs for EEGNet vs

# **About SpeechBrain**
- Website: https://speechbrain.github.io/
- Code: https://github.com/speechbrain/speechbrain/
- HuggingFace: https://huggingface.co/speechbrain/


# **Citing SpeechBrain**
Please, cite SpeechBrain if you use it for your research or business.

```bibtex
@misc{speechbrain,
  title={{SpeechBrain}: A General-Purpose Speech Toolkit},
  author={Mirco Ravanelli and Titouan Parcollet and Peter Plantinga and Aku Rouhe and Samuele Cornell and Loren Lugosch and Cem Subakan and Nauman Dawalatabad and Abdelwahab Heba and Jianyuan Zhong and Ju-Chieh Chou and Sung-Lin Yeh and Szu-Wei Fu and Chien-Feng Liao and Elena Rastorgueva and François Grondin and William Aris and Hwidong Na and Yan Gao and Renato De Mori and Yoshua Bengio},
  year={2021},
  eprint={2106.04624},
  archivePrefix={arXiv},
  primaryClass={eess.AS},
  note={arXiv:2106.04624}
}
```